In [2]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [3]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [4]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Chennai')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [27]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude, pickup_location_hex_8,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, map_riders, drop_location_hex_8
from orders.order_logs_snapshot
where yyyymmdd >= '20230206'
and yyyymmdd <= '20230210'
and city_name in ('Chennai')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [28]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [29]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,pickup_location_hex_8,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,map_riders,drop_location_hex_8
0,63e5a7c9446060795b9f024b,0.624,88618c4f3bfffff,13.035661,88618c4893fffff,80.212395,Chennai,88618c4893fffff,13.067410,80.235214,074121,07,0730,Link,07:59,5,20230210,dropped,None,"[""5d9f5c85941a7d1c51df0de9"",""62b94f2b21385bd70...",88618c4f3bfffff
1,63e5a7ff223272401156a6c2,2.208,88618c48d3fffff,13.052345,88618c4f3dfffff,80.257487,Chennai,88618c48d3fffff,13.052345,80.257487,074215,07,0730,Link,07:59,5,20230210,customerCancelled,Taking longer than expected,"[""5fed6996857d1be866222934"",""612532fd34b93d8b1...",88618c48dbfffff
2,63e5a80213cf522e20a7ff9c,NaN,88618c48b3fffff,13.090466,88618c48b3fffff,80.246178,Chennai,88618c48b3fffff,13.090466,80.246178,074218,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""610cd77d742aee80255c78dd"",""62d0651a852b21803...",88618eb257fffff
3,63e5a8062f0d965b714d00c5,3.789,88618eb211fffff,13.125811,88618eb2c7fffff,80.219780,Chennai,88618eb2c7fffff,13.126427,80.193617,074222,07,0730,Link,07:59,5,20230210,dropped,None,"[""635c9a0109d2f275d6c77f5c"",""6201124497de5609c...",88618eb211fffff
4,63e5a819275abd141cf8c57e,NaN,88618c4d59fffff,13.048904,88618c4d59fffff,80.206924,Chennai,88618c4d59fffff,13.049013,80.206924,074241,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""6389ed2c7948c416fe132029"",""5e6a386f0cd711af2...",88618c4f13fffff


In [30]:
df = df.loc[:,~df.columns.duplicated()].copy()
df_cluster_copy = df_cluster_copy.rename(columns ={'hex_id':'pickup_location_hex_8'})
df = df.merge(df_cluster_copy[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_grouped = df[df['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['cluster'])['order_id'].nunique().reset_index()
cluster_grouped = cluster_grouped.sort_values(by = 'order_id', ascending=False)
cluster_grouped.head(20)

,cluster,order_id
90,Nungambakkam,6213
127,T Nagar,5816
156,kilpauk,4594
12,Anna Nagar,4026
130,Teynampet,3686
84,Mylapore,3484
107,Porur,3412
133,Thiruvanmiyur,3387
113,Royapettah,3351
45,Egmore,3288


In [41]:
chennai_zone = pd.read_csv('chennai_small_zone_demand.csv')
chennai_zone_hexes = chennai_zone['pickup_location_hex_8'].unique().tolist()

In [42]:
#Filter only on the hexes needed
df_filter_level1 = df[df['pickup_location_hex_8'].isin(chennai_zone_hexes)]
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [43]:
print(chennai_zone_hexes)

['88618c4f07fffff', '88618c4f05fffff', '88618c48d1fffff', '88618c4ab3fffff', '88618c4f29fffff', '88618c4f65fffff', '88618c48d3fffff', '88618c4f61fffff', '88618c4f3dfffff', '88618c4f2bfffff', '88618c4f2dfffff', '88618c4f25fffff', '88618c4f35fffff', '88618c4abbfffff', '88618c48ddfffff', '88618c4f63fffff', '88618c4f69fffff', '88618c4d49fffff', '88618c4d4dfffff', '88618c48d5fffff', '88618c4d41fffff', '88618c4f6bfffff', '88618c48dbfffff', '88618c4f67fffff', '88618c4899fffff', '88618c4ab7fffff', '88618c4f01fffff', '88618c4f47fffff', '88618c4f27fffff', '88618c4f03fffff', '88618c48c3fffff', '88618c4f39fffff', '88618c48d9fffff', '88618c4f6dfffff', '88618c4f45fffff', '88618c4ab1fffff', '88618c48d7fffff', '88618c4f21fffff', '88618c4a97fffff', '88618c4f23fffff', '88618c4f4dfffff', '88618c4f0dfffff', '88618c489bfffff', '88618c48cbfffff', '88618c4f15fffff', '88618c4a91fffff', '88618c4f09fffff', '88618c4a93fffff', '88618c4f0bfffff']


In [44]:
df_filter_level1[df_filter_level1['drop_location_hex_8'].isin(chennai_zone_hexes)].shape[0]/df_filter_level1.shape[0]

0.5556431157189647

In [35]:
q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230206'
and yyyymmdd <= '20230210'
and service_obj_service_name = 'Link'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and pickup_location_hex_8 in ('88618c4f07fffff', '88618c4f05fffff', '88618c48d1fffff', '88618c4ab3fffff', '88618c4f29fffff', '88618c4f65fffff', '88618c48d3fffff',
 '88618c4f61fffff', '88618c4895fffff', '88618c4f3dfffff', '88618c4f2bfffff', '88618c4f2dfffff', '88618c4f25fffff', '88618c4f35fffff', '88618c4abbfffff',
   '88618c48ddfffff', '88618c4f63fffff', '88618c4f69fffff', '88618c4d49fffff', '88618c4d4dfffff', '88618c48d5fffff', '88618c4891fffff', '88618c4d41fffff',
     '88618c4883fffff', '88618c4f6bfffff', '88618c488bfffff', '88618c48dbfffff', '88618c4f67fffff', '88618c489dfffff', '88618c4899fffff', '88618c4ab7fffff',
       '88618c4f01fffff', '88618c4881fffff', '88618c4f47fffff', '88618c4f27fffff', '88618c4f03fffff', '88618c48c3fffff', '88618c4f39fffff', '88618c48d9fffff',
         '88618c4f6dfffff', '88618c4f45fffff', '88618c4ab1fffff', '88618c48d7fffff', '88618c4f21fffff', '88618c4a97fffff', '88618c4f23fffff', '88618c4f4dfffff',
           '88618c4f0dfffff', '88618c489bfffff', '88618c48cbfffff', '88618c4f15fffff', '88618c4a91fffff', '88618c4f09fffff', '88618c4a93fffff', '88618c4f0bfffff')
"""

df_apr_copy = pd.read_sql(q, connection)


In [36]:
#View the dataset
df_apr = df_apr_copy.copy()
df_cluster_copy = df_cluster_copy.rename(columns = {'pickup_location_hex_8':'captain_location_hex_8'})
df_apr = df_apr.merge(df_cluster_copy[['captain_location_hex_8', 'cluster']], on = 'captain_location_hex_8', how = 'left')
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,cluster
0,63e57f9f223272401156748e,NaN,88618c4883fffff,13.075677,80.259281,Chennai,88618c4883fffff,88618c4d4bfffff,13.054784,80.223782,13.077354,80.261459,044959,04,0445,Link,04:59,5,20230210,new,None,1675984810609,rider_reject,5d2f703001197b4e4d1bf690,None,88618c4883fffff,Egmore Metro
1,63e578c52f0d965b714ccb44,NaN,88618c4891fffff,13.069769,80.242405,Chennai,88618c4899fffff,88618c4f3dfffff,13.039286,80.228877,13.060613,80.251144,042045,04,0415,Link,04:00,5,20230210,new,None,1675983115546,rider_busy,637fb30e2519f5c9181479c0,None,88618c4899fffff,Egmore
2,63e581dc2f0d965b714ccfdb,NaN,88618c4883fffff,13.077089,80.260362,Chennai,88618c4883fffff,88618c49dbfffff,13.101779,80.291079,13.079389,80.258842,045932,04,0445,Link,04:59,5,20230210,onTheWay,None,1675985378825,accepted,62f49d20f621dae68d16b813,None,88618c4883fffff,Egmore Metro
3,63e579f722327240115671e7,NaN,88618c4883fffff,13.079278,80.262147,Chennai,88618c4883fffff,88618c48ddfffff,13.054091,80.269838,13.077242,80.260399,042551,04,0415,Link,04:00,5,20230210,onTheWay,None,1675983354769,accepted,61b97bfdf3de6f56404d25b4,None,88618c4883fffff,Egmore Metro
4,63e578c52f0d965b714ccb44,NaN,88618c4f21fffff,13.047633,80.244936,Chennai,88618c4899fffff,88618c4f3dfffff,13.039286,80.228877,13.060613,80.251144,042045,04,0415,Link,04:00,5,20230210,new,None,1675983052875,rider_reject,63d6de237cddcde38ac5e4d9,None,88618c4899fffff,T Nagar


In [38]:
#Create a count column
df_apr['total_pings'] = 1

#Merge the cluster to apr
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)

#Get the pings sent to captains by location
df_apr_filter = df_apr[df_apr['hour'].isin(['17', '18', '19', '20', '21'])]
df_apr_grouped = df_apr_filter.groupby(['cluster'])['total_pings', 'accepted_pings'].sum().reset_index()
df_apr_grouped = df_apr_grouped.sort_values(by = ['total_pings'], ascending= [False])
df_apr_grouped['APR'] = round(df_apr_grouped['accepted_pings']/df_apr_grouped['total_pings'], 4)
df_apr_grouped.head()

,cluster,total_pings,accepted_pings,APR
47,T Nagar,14931,5455,0.3653
30,Nungambakkam,14115,4653,0.3296
48,Teynampet,12279,3749,0.3053
42,Royapettah,10582,3049,0.2881
29,Mylapore,9663,3239,0.3352


In [45]:
print(len(chennai_zone_hexes))

49


In [40]:
df_apr_grouped.to_csv('APR_chennai_zone.csv', index = False)

In [29]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [30]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [31]:
df_filter_level1[df_filter_level1['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['hour'])['accept_to_pickup_distance'].mean().reset_index()

,hour,accept_to_pickup_distance
0,17,1.219665
1,18,1.216394
2,19,1.015161
3,20,0.917163
4,21,0.973049


In [32]:
df_filter_level1[(df_filter_level1['hour'].isin(['17', '18', '19', '20', '21'])) &
 (df_filter_level1['final_order_status'] == 'dropped')].groupby(['hour'])['accept_to_pickup_distance'].mean().reset_index()

,hour,accept_to_pickup_distance
0,17,1.150457
1,18,1.108866
2,19,0.924063
3,20,0.869657
4,21,0.945395


In [46]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,pickup_location_hex_8,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,map_riders,drop_location_hex_8,cluster,second,minute,date
0,63e5a7c9446060795b9f024b,0.624,88618c4f3bfffff,13.035661,88618c4893fffff,80.212395,Chennai,88618c4893fffff,13.067410,80.235214,074121,07,0730,Link,07:59,5,20230210,dropped,None,"[""5d9f5c85941a7d1c51df0de9"",""62b94f2b21385bd70...",88618c4f3bfffff,Shenoy Nagar,21,41,10
1,63e5a7ff223272401156a6c2,2.208,88618c48d3fffff,13.052345,88618c4f3dfffff,80.257487,Chennai,88618c48d3fffff,13.052345,80.257487,074215,07,0730,Link,07:59,5,20230210,customerCancelled,Taking longer than expected,"[""5fed6996857d1be866222934"",""612532fd34b93d8b1...",88618c48dbfffff,T Nagar,15,42,10
2,63e5a80213cf522e20a7ff9c,NaN,88618c48b3fffff,13.090466,88618c48b3fffff,80.246178,Chennai,88618c48b3fffff,13.090466,80.246178,074218,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""610cd77d742aee80255c78dd"",""62d0651a852b21803...",88618eb257fffff,kilpauk,18,42,10
3,63e5a8062f0d965b714d00c5,3.789,88618eb211fffff,13.125811,88618eb2c7fffff,80.219780,Chennai,88618eb2c7fffff,13.126427,80.193617,074222,07,0730,Link,07:59,5,20230210,dropped,None,"[""635c9a0109d2f275d6c77f5c"",""6201124497de5609c...",88618eb211fffff,Korattur,22,42,10
4,63e5a819275abd141cf8c57e,NaN,88618c4d59fffff,13.048904,88618c4d59fffff,80.206924,Chennai,88618c4d59fffff,13.049013,80.206924,074241,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""6389ed2c7948c416fe132029"",""5e6a386f0cd711af2...",88618c4f13fffff,Vadapalani Fortis Hospital,41,42,10


In [47]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,pickup_location_hex_8,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,map_riders,drop_location_hex_8,cluster,second,minute,date,20_sec_batch,count
0,63e5a7c9446060795b9f024b,0.624,88618c4f3bfffff,13.035661,88618c4893fffff,80.212395,Chennai,88618c4893fffff,13.067410,80.235214,074121,07,0730,Link,07:59,5,20230210,dropped,None,"[""5d9f5c85941a7d1c51df0de9"",""62b94f2b21385bd70...",88618c4f3bfffff,Shenoy Nagar,21.0,41.0,10,20-40,1
1,63e5a7ff223272401156a6c2,2.208,88618c48d3fffff,13.052345,88618c4f3dfffff,80.257487,Chennai,88618c48d3fffff,13.052345,80.257487,074215,07,0730,Link,07:59,5,20230210,customerCancelled,Taking longer than expected,"[""5fed6996857d1be866222934"",""612532fd34b93d8b1...",88618c48dbfffff,T Nagar,15.0,42.0,10,0-20,1
2,63e5a80213cf522e20a7ff9c,NaN,88618c48b3fffff,13.090466,88618c48b3fffff,80.246178,Chennai,88618c48b3fffff,13.090466,80.246178,074218,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""610cd77d742aee80255c78dd"",""62d0651a852b21803...",88618eb257fffff,kilpauk,18.0,42.0,10,0-20,1
3,63e5a8062f0d965b714d00c5,3.789,88618eb211fffff,13.125811,88618eb2c7fffff,80.219780,Chennai,88618eb2c7fffff,13.126427,80.193617,074222,07,0730,Link,07:59,5,20230210,dropped,None,"[""635c9a0109d2f275d6c77f5c"",""6201124497de5609c...",88618eb211fffff,Korattur,22.0,42.0,10,20-40,1
4,63e5a819275abd141cf8c57e,NaN,88618c4d59fffff,13.048904,88618c4d59fffff,80.206924,Chennai,88618c4d59fffff,13.049013,80.206924,074241,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""6389ed2c7948c416fe132029"",""5e6a386f0cd711af2...",88618c4f13fffff,Vadapalani Fortis Hospital,41.0,42.0,10,40-60,1


In [48]:
#Check the number of rides coming from each hex
df_grouped = df.groupby(['pickup_location_hex_8'])['count'].sum().reset_index()

#Filter out those hexes where atleast 1000 rides have come in the last two weeks
df_grouped = df_grouped[df_grouped['count'] >= 100].reset_index(drop = True)

#Add a column to keep for analysis
df_grouped['keep_for_analysis'] = 'yes'
df_grouped.head()

,pickup_location_hex_8,count,keep_for_analysis
0,88618c4081fffff,327,yes
1,88618c4083fffff,201,yes
2,88618c4085fffff,241,yes
3,88618c4087fffff,638,yes
4,88618c408bfffff,956,yes


In [49]:
#Merge the number of rides to the original df
df = df.merge(df_grouped[['pickup_location_hex_8', 'keep_for_analysis']], on = 'pickup_location_hex_8', how = 'left')
df['keep_for_analysis'] = df['keep_for_analysis'].fillna('no')
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,pickup_location_hex_8,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,map_riders,drop_location_hex_8,cluster,second,minute,date,20_sec_batch,count,keep_for_analysis
0,63e5a7c9446060795b9f024b,0.624,88618c4f3bfffff,13.035661,88618c4893fffff,80.212395,Chennai,88618c4893fffff,13.067410,80.235214,074121,07,0730,Link,07:59,5,20230210,dropped,None,"[""5d9f5c85941a7d1c51df0de9"",""62b94f2b21385bd70...",88618c4f3bfffff,Shenoy Nagar,21.0,41.0,10,20-40,1,yes
1,63e5a7ff223272401156a6c2,2.208,88618c48d3fffff,13.052345,88618c4f3dfffff,80.257487,Chennai,88618c48d3fffff,13.052345,80.257487,074215,07,0730,Link,07:59,5,20230210,customerCancelled,Taking longer than expected,"[""5fed6996857d1be866222934"",""612532fd34b93d8b1...",88618c48dbfffff,T Nagar,15.0,42.0,10,0-20,1,yes
2,63e5a80213cf522e20a7ff9c,NaN,88618c48b3fffff,13.090466,88618c48b3fffff,80.246178,Chennai,88618c48b3fffff,13.090466,80.246178,074218,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""610cd77d742aee80255c78dd"",""62d0651a852b21803...",88618eb257fffff,kilpauk,18.0,42.0,10,0-20,1,yes
3,63e5a8062f0d965b714d00c5,3.789,88618eb211fffff,13.125811,88618eb2c7fffff,80.219780,Chennai,88618eb2c7fffff,13.126427,80.193617,074222,07,0730,Link,07:59,5,20230210,dropped,None,"[""635c9a0109d2f275d6c77f5c"",""6201124497de5609c...",88618eb211fffff,Korattur,22.0,42.0,10,20-40,1,yes
4,63e5a819275abd141cf8c57e,NaN,88618c4d59fffff,13.048904,88618c4d59fffff,80.206924,Chennai,88618c4d59fffff,13.049013,80.206924,074241,07,0730,Link,07:59,5,20230210,customerCancelled,order cancelled before rider accepted,"[""6389ed2c7948c416fe132029"",""5e6a386f0cd711af2...",88618c4f13fffff,Vadapalani Fortis Hospital,41.0,42.0,10,40-60,1,yes


In [50]:
#Filter the df only on hexes to keep
df_filter = df[df['keep_for_analysis'] == 'yes']

In [51]:
#Groupby on hex to see number of orders coming in 15 and 20 second batches
df_hex_grouped = df_filter.groupby(['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])['count'].sum().reset_index()
df_hex_grouped = df_hex_grouped.sort_values(by = ['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])
df_hex_grouped.head()

,pickup_location_hex_8,date,hour,minute,20_sec_batch,count
0,88618c4081fffff,06,00,0.0,0-20,0
1,88618c4081fffff,06,00,0.0,20-40,0
2,88618c4081fffff,06,00,0.0,40-60,0
3,88618c4081fffff,06,00,1.0,0-20,0
4,88618c4081fffff,06,00,1.0,20-40,0


In [52]:
#Identify high demand hexes
df_hex_grouped_filter2 = df_hex_grouped[df_hex_grouped['count'] >= 1]
count_df = pd.DataFrame(df_hex_grouped_filter2['pickup_location_hex_8'].value_counts().reset_index())
count_df.columns = ['pickup_location_hex_8', 'count']
count_df.head(20)

,pickup_location_hex_8,count
0,88618c4d0bfffff,5370
1,88618c4885fffff,4497
2,88618c4f1bfffff,4360
3,88618c4883fffff,3874
4,88618c4f37fffff,3069
5,88618c4d5dfffff,3020
6,88618c48d3fffff,2922
7,88618c48d5fffff,2881
8,88618c4f25fffff,2724
9,88618c4d4dfffff,2715


In [13]:
df[['customer_location_hex_8', 'count']].to_csv('jaipur_hexes.csv', index = False)

In [53]:
#Look at the average demand for each minute
df_hex_grouped['time'] = (pd.to_datetime(df_hex_grouped['hour'].astype(str) + ':' + df_hex_grouped['minute'].astype(int).astype(str), format='%H:%M').dt.time)
df_hex_grouped.head()

,pickup_location_hex_8,date,hour,minute,20_sec_batch,count,time
0,88618c4081fffff,06,00,0.0,0-20,0,00:00:00
1,88618c4081fffff,06,00,0.0,20-40,0,00:00:00
2,88618c4081fffff,06,00,0.0,40-60,0,00:00:00
3,88618c4081fffff,06,00,1.0,0-20,0,00:01:00
4,88618c4081fffff,06,00,1.0,20-40,0,00:01:00


In [54]:
#Filter on the hexes identified as high demand
high_demand_hexes = ['883da219ddfffff', '883da219d1fffff', '883da219d3fffff', '883da218a5fffff', '883da219d5fffff', '883da219d7fffff', '883da21999fffff',
 '883da2198bfffff', '883da2199dfffff', '883da219b9fffff', '883da21995fffff', '883da219b3fffff', '883da20a6dfffff', '883da219c7fffff']
hexes = []
for x in high_demand_hexes:
    hex_ids = h3.k_ring(x, 1)
    hexes.append(hex_ids)

hexes = [item for sublist in hexes for item in sublist]

In [55]:
#Get the total number of rides per hour
df_hex_high_demand = df_hex_grouped[df_hex_grouped['pickup_location_hex_8'].isin(chennai_zone_hexes)]
df_level_1_grouped = round(df_hex_high_demand.groupby(['date', 'hour'])['count'].sum().reset_index(), 0)
df_level_1_grouped = df_level_1_grouped.rename(columns = {'count':'rides_per_hour'})
df_level_1_grouped['rides_per_20_sec'] = df_level_1_grouped['rides_per_hour'].apply(lambda x: round(x/180, 2))
df_level_1_grouped

,date,hour,rides_per_hour,rides_per_20_sec
0,06,00,107,0.59
1,06,01,50,0.28
2,06,02,32,0.18
3,06,03,64,0.36
4,06,04,64,0.36
...,...,...,...,...
115,10,19,1439,7.99
116,10,20,1116,6.20
117,10,21,898,4.99
118,10,22,696,3.87


In [56]:
df_level_1_grouped[df_level_1_grouped['hour'].isin(['08', '09', '10', '11', '17', '18', '19', '20', '21'])]

,date,hour,rides_per_hour,rides_per_20_sec
8,06,08,1143,6.35
9,06,09,1636,9.09
10,06,10,1170,6.50
11,06,11,741,4.12
17,06,17,1115,6.19
18,06,18,1579,8.77
19,06,19,1195,6.64
20,06,20,947,5.26
21,06,21,694,3.86
32,07,08,855,4.75


In [57]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].median()

hour
00    0.75
01    0.29
02    0.22
03    0.23
04    0.36
05    0.61
06    1.13
07    2.53
08    5.18
09    7.75
10    6.14
11    4.13
12    4.19
13    4.66
14    4.09
15    4.38
16    5.11
17    6.49
18    8.77
19    6.86
20    5.34
21    4.08
22    3.40
23    1.74
Name: rides_per_20_sec, dtype: float64

In [44]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].mean()

hour
00     1.668
01     0.798
02     0.582
03     0.884
04     1.182
05     1.774
06     3.138
07     8.080
08    13.444
09    18.252
10    13.762
11     9.790
12     9.938
13    10.802
14    10.324
15    10.538
16    11.886
17    14.304
18    17.366
19    15.298
20    12.644
21    10.122
22     8.204
23     4.096
Name: rides_per_20_sec, dtype: float64

In [44]:
df_hex_high_demand_grouped.to_csv('demand_chennai.csv', index = False)

In [43]:
df_hex_high_demand_grouped = df_hex_high_demand.groupby(['customer_location_hex_8'])['count'].sum().reset_index()
df_hex_high_demand_grouped

,customer_location_hex_8,count
0,88618c4881fffff,1095
1,88618c4883fffff,3407
2,88618c4885fffff,3705
3,88618c4887fffff,1339
4,88618c4889fffff,2186
5,88618c488bfffff,1321
6,88618c488dfffff,369
7,88618c4891fffff,1183
8,88618c4893fffff,816
9,88618c4895fffff,1343
